In [1]:
###Load modules
#!pip install webdriver-manager
#!pip install awscli
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]') #identifies the location of the EIN element
inputElement.send_keys('0') #sends the "0" as the search value for EIN 
inputElement1 = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]').click() #instatiates the click of the search
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table

for row in table.find_elements(By.CSS_SELECTOR,'tr'):
    if row.find_elements(By.XPATH, './parent::thead'):
        continue  # Skip rows within the <thead> section
    cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
5,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
6,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
7,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
8,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
9,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA


In [23]:
###LOAD THE FILE INTO S3####
# prepare csv file name 
pathname = 's3://ia-m10-assn-bhushan-babar/' #specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True) 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://ia-m10-assn-bhushan-babar/charities_bureau_scrape_20240414141316.csv


## Check bucket Content 

In [6]:
# Custom function to inspect and validate the data uploaded into an S3 bucket
def inspect_s3_contents(bucket_name):
    import boto3

    session = boto3.Session(
        region_name='us-east-1'
    )
    
    s3 = session.client('s3')

    # Retrieve objects from the specified bucket
    response = s3.list_objects_v2(Bucket=bucket_name)

    # Check if the bucket contains any objects
    if 'Contents' in response:
        print("The contents of the '{}' bucket are as follows:".format(bucket_name))
        # Iterate through each object in the bucket and print its key
        for s3_object in response['Contents']:
            print(s3_object['Key'])
    else:
        print("The bucket '{}' is empty.".format(bucket_name))

In [7]:
inspect_s3_contents('ia-m10-assn-bhushan-babar')

The contents of the 'ia-m10-assn-bhushan-babar' bucket are as follows:
charities_bureau_scrape_fixed_row20240413203010.csv


# Part 2

In [8]:
#Import libraries
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [13]:
####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(By.XPATH,"/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

all_data = []  # Empty list to store data from each page
while True:
    try:
        table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
        sleep(1)
        next_button = browser.find_element(By.XPATH, '/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a[text()="Next"]')
        sleep(1)
        for row in table.find_elements(By.CSS_SELECTOR,'tr'):
            if row.find_elements(By.XPATH, './parent::thead'):
                continue  # Skip rows within the <thead> section
            all_data.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])
        next_button.click()
        sleep(4)
    except NoSuchElementException:
        table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
        sleep(1)
        for row in table.find_elements(By.CSS_SELECTOR,'tr'):
            if row.find_elements(By.XPATH, './parent::thead'):
                continue  # Skip rows within the <thead> section
            all_data.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])
        # Handle the case where "Next" button is not found (reached last page)
        print("Checked all pages.")
        break
#update dataframe with header 
df = pd.DataFrame(all_data, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

Checked all pages.


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
...,...,...,...,...,...,...
95,University of Virginia Health Foundtion,40-44-88,412097394,NFP,CHARLOTTESVILLE,VA
96,Violin Player,41-40-19,270773158,NFP,LONDONDERRY,NH
97,"William A. Epps Community Center, Inc.",40-91-11,861074714,NFP,STATEN ISLAND,NY
98,WORLD SOCIETY OF CZESTOCHOWA JEWS AND THEIR DE...,40-46-49,205101779,NFP,NEW YORK,NY


In [11]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://ia-m10-assn-bhushan-babar/' #specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_allpagedata_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True) 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://ia-m10-assn-bhushan-babar/charities_bureau_scrape_allpagedata_20240414202117.csv


In [12]:
inspect_s3_contents('ia-m10-assn-bhushan-babar')

The contents of the 'ia-m10-assn-bhushan-babar' bucket are as follows:
charities_bureau_scrape_allpagedata_20240414202117.csv
charities_bureau_scrape_fixed_row20240413203010.csv
